In [12]:
import hopsworks
import pandas as pd
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import joblib
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

In [2]:
project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/195734
Connected. Call `.close()` to terminate connection gracefully.


In [3]:
wine_fg = fs.get_feature_group(name="wine", version=1)
query = wine_fg.select_all()
feature_view = fs.get_or_create_feature_view(
    name="wine",
    version=1,
    description="Read from Wine Quality dataset",
    labels=["quality"],
    query=query,
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/195734/fs/195653/fv/wine/version/1


In [10]:
X_train, X_test, y_train, y_test = feature_view.train_test_split(0.2)

rf = RandomForestRegressor(n_estimators=100)
rf.fit(X_train, y_train.values.ravel())

Finished: Reading data from Hopsworks, using Hive (2.44s) 


RandomForestRegressor()

In [11]:
y_pred = rf.predict(X_test)

rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"RF RMSE: {rmse}")

RF RMSE: 0.7155734943858023


### Upload to Hopsworks model registry

In [ ]:
mr = project.get_model_registry()

model_dir = "wine_model"

if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)

joblib.dump(rf, model_dir + "wine_model.pkl")

input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema, output_schema)

wine_model = mr.python.create_model(
    name = "wine_model",
    metrics={"accuracy":rmse},
    model_schema=model_schema=,
    description = "Wine Quality Regression Model",
)